## Import

In [4]:
!pip install tensorboard

  Using cached tensorboard-2.15.1-py3-none-any.whl (5.5 MB)
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached grpcio-1.60.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)
  Using cached google_auth-2.26.2-py2.py3-none-any.whl (186 kB)
  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl (24 kB)
  Using cached Markdown-3.5.2-py3-none-any.whl (103 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl (6.6 MB)
  Using cached cachetools-5.3.2-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)


In [5]:
!pip install more_itertools

  Using cached more_itertools-10.2.0-py3-none-any.whl (57 kB)


In [25]:
import json
from datetime import datetime
import torch.nn as nn

from args import get_parser
from utils import *
from mtad_gat import MTAD_GAT
from prediction import Predictor
from training import Trainer

In [26]:
class Args:
    def __init__(self):
        # Data params
        self.dataset = "IVECO"
        self.resample_rate = 1.0
        self.cut = 1.0
        self.scaler = 'standard'
        self.train_test_split = 0.7
        self.no_anomaly_train = True
        self.group = "1-1"
        self.lookback = 10
        self.normalize = True
        self.spectral_residual = False
        self.spec_res = False

        # Model params
        self.kernel_size = 7
        self.use_gatv2 = True
        self.feat_gat_embed_dim = None
        self.time_gat_embed_dim = None
        self.gru_n_layers = 1
        self.gru_hid_dim = 150
        self.fc_n_layers = 3
        self.fc_hid_dim = 50
        self.recon_n_layers = 10
        self.recon_hid_dim = 1500
        self.alpha = 0.2
        self.reduce_dimensionality = True
        self.n_features_reduced = 30
        self.use_tcn = True
        self.use_vae = True

        # Train params
        self.epochs = 600
        self.val_split = 0.05
        self.bs = 256
    
        self.init_lr = 1e-3
        self.shuffle_dataset = True
        self.dropout = 0.1
        self.use_cuda = True
        self.print_every = 10
        self.log_tensorboard = True

        # Predictor params
        self.scale_scores = False
        self.use_mov_av = False
        self.gamma = 1.0
        self.level = None
        self.q = None
        self.dynamic_pot = False

        # Other
        self.comment = ""




## Training

In [27]:
id = datetime.now().strftime("%d%m%Y_%H%M%S")

parser = get_parser()
args = Args()

dataset = args.dataset
window_size = args.lookback # default 100
spec_res = args.spec_res 
normalize = args.normalize # default True
n_epochs = args.epochs #default 30
batch_size = args.bs # default 256
init_lr = args.init_lr
val_split = args.val_split
shuffle_dataset = args.shuffle_dataset #default true
use_cuda = args.use_cuda
print_every = args.print_every
log_tensorboard = args.log_tensorboard
group_index = args.group[0]
index = args.group[2:]
args_summary = str(args.__dict__)
print(args_summary)

if dataset == 'SMD':
    output_path = f'output/SMD/{args.group}'
    (x_train, _), (x_test, y_test) = get_data(f"machine-{group_index}-{index}", normalize=normalize)
elif dataset in ['MSL', 'SMAP', 'SWAT', 'SKAB', 'WADI','METRO', 'ACT', 'IVECO']:
    output_path = f'output/{dataset}'
    (x_train, _), (x_test, y_test) = get_data(dataset, normalize=normalize)
else:
    raise Exception(f'Dataset "{dataset}" not available.')

log_dir = f'{output_path}/logs'
if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
save_path = f"{output_path}/{id}"

x_train = torch.from_numpy(x_train).float()
x_test = torch.from_numpy(x_test).float()
n_features = x_train.shape[1]

target_dims = get_target_dims(dataset)
if target_dims is None:
    out_dim = n_features
    print(f"Will forecast and reconstruct all {n_features} input features")
elif type(target_dims) == int:
    print(f"Will forecast and reconstruct input feature: {target_dims}")
    out_dim = 1
else:
    print(f"Will forecast and reconstruct input features: {target_dims}")
    out_dim = len(target_dims)

train_dataset = SlidingWindowDataset(x_train, window_size, target_dims)
test_dataset = SlidingWindowDataset(x_test, window_size, target_dims)

train_loader, val_loader, test_loader = create_data_loaders(
    train_dataset, batch_size, val_split, shuffle_dataset, test_dataset=test_dataset
)

model = MTAD_GAT(
    n_features,
    window_size,
    out_dim,
    kernel_size=args.kernel_size,
    use_gatv2=args.use_gatv2,
    feat_gat_embed_dim=args.feat_gat_embed_dim,
    time_gat_embed_dim=args.time_gat_embed_dim,
    gru_n_layers=args.gru_n_layers,
    gru_hid_dim=args.gru_hid_dim,
    forecast_n_layers=args.fc_n_layers,
    forecast_hid_dim=args.fc_hid_dim,
    recon_n_layers=args.recon_n_layers,
    recon_hid_dim=args.recon_hid_dim,
    dropout=args.dropout,
    alpha=args.alpha,
    reduce_dimensionality = args.reduce_dimensionality,
    n_features_reduced = args.n_features_reduced,
    use_tcn = args.use_tcn,
    use_vae = args.use_vae
)

optimizer = torch.optim.Adam(model.parameters(), lr=args.init_lr)
forecast_criterion = nn.MSELoss()
recon_criterion = nn.MSELoss()

trainer = Trainer(
    model,
    optimizer,
    window_size,
    n_features,
    target_dims,
    n_epochs,
    batch_size,
    init_lr,
    forecast_criterion,
    recon_criterion,
    use_cuda,
    save_path,
    log_dir,
    print_every,
    log_tensorboard,
    args_summary
)

trainer.fit(train_loader, val_loader)

plot_losses(trainer.losses, save_path=save_path, plot=False)



{'dataset': 'IVECO', 'resample_rate': 1.0, 'cut': 1.0, 'scaler': 'standard', 'train_test_split': 0.7, 'no_anomaly_train': True, 'group': '1-1', 'lookback': 10, 'normalize': True, 'spectral_residual': False, 'spec_res': False, 'kernel_size': 7, 'use_gatv2': True, 'feat_gat_embed_dim': None, 'time_gat_embed_dim': None, 'gru_n_layers': 1, 'gru_hid_dim': 150, 'fc_n_layers': 3, 'fc_hid_dim': 50, 'recon_n_layers': 10, 'recon_hid_dim': 1500, 'alpha': 0.2, 'reduce_dimensionality': True, 'n_features_reduced': 30, 'use_tcn': True, 'use_vae': True, 'epochs': 1000, 'val_split': 0.05, 'bs': 256, 'init_lr': 0.001, 'shuffle_dataset': True, 'dropout': 0.1, 'use_cuda': True, 'print_every': 10, 'log_tensorboard': True, 'scale_scores': False, 'use_mov_av': False, 'gamma': 1.0, 'level': None, 'q': None, 'dynamic_pot': False, 'comment': ''}
load data of: IVECO
train:  0 None
test:  0 None
Data normalized
Data normalized
train set shape:  (2173, 195)
test set shape:  (6832, 195)
test set label shape:  (6832

## Testing

In [28]:
test_loss = trainer.evaluate(test_loader)
print(f"Test forecast loss (mean): {test_loss[0]:.5f}")
print(f"Test reconstruction loss (mean): {test_loss[1]:.5f}")
print(f"Test total loss (mean): {test_loss[2]:.5f}")

# Some suggestions for POT args
level_q_dict = {
    "SMAP": (0.90, 0.005),
    "MSL": (0.90, 0.001),
    "SWAT": (0.90, 0.001),
    "WADI": (0.90, 0.001),
    "METRO": (0.90, 0.001),
    "ACT": (0.90, 0.001),
    "SKAB": (0.90, 0.001),
    "SMD-1": (0.9950, 0.001),
    "SMD-2": (0.9925, 0.001),
    "SMD-3": (0.9999, 0.001),
    'IVECO': (0.90, 0.001)
}
key = "SMD-" + args.group[0] if args.dataset == "SMD" else args.dataset
level, q = level_q_dict[key]
if args.level is not None:
    level = args.level
if args.q is not None:
    q = args.q

# Some suggestions for Epsilon args
reg_level_dict = {"SMAP": 0,"SWAT": 0,"WADI": 0, 'IVECO': 0, "METRO": 0,"ACT": 0,"SKAB":0, "MSL": 0, "SMD-1": 1, "SMD-2": 1, "SMD-3": 1}
key = "SMD-" + args.group[0] if dataset == "SMD" else dataset
reg_level = reg_level_dict[key]

trainer.load(f"{save_path}/model.pt")
prediction_args = {
    'dataset': dataset,
    "target_dims": target_dims,
    'scale_scores': args.scale_scores,
    "level": level,
    "q": q,
    'dynamic_pot': args.dynamic_pot,
    "use_mov_av": args.use_mov_av,
    "gamma": args.gamma,
    "reg_level": reg_level,
    "save_path": save_path,
}
best_model = trainer.model
predictor = Predictor(
    best_model,
    window_size,
    n_features,
    prediction_args,
)

label = y_test[window_size:] if y_test is not None else None
predictor.predict_anomalies(x_train, x_test, label)

# Save config
args_path = f"{save_path}/config.txt"
with open(args_path, "w") as f:
    json.dump(args.__dict__, f, indent=2)

Test forecast loss: 1.66959
Test reconstruction loss: 1.66231
Test total loss: 3.33190
Predicting and calculating anomaly scores..


100%|██████████| 9/9 [00:00<00:00, 69.75it/s]


Predicting and calculating anomaly scores..


100%|██████████| 27/27 [00:00<00:00, 72.58it/s]
/root/mtad-gat-pytorch/prediction.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_pred_df[f"A_Pred_{i}"] = train_feature_anom_preds
/root/mtad-gat-pytorch/prediction.py:149: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_pred_df[f"A_Pred_{i}"] = test_feature_anom_preds
/root/mtad-gat-pytorch/prediction.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

Running POT with q=0.001, level=0.9..
Initial threshold : 0.12991337
Number of peaks : 216
Grimshaw maximum log-likelihood estimation ... [done]
	γ = -0.014038187451660633
	σ = 0.02247774962435588
	L = 606.8017907297382
Extreme quantile (probability = 0.001): 0.23012295167414576


100%|██████████| 6822/6822 [00:00<00:00, 2039018.16it/s]


0
6822
Finding best f1-score by searching for threshold..
Results using epsilon method:
 {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'TP': 0, 'TN': 4093, 'FP': 2728, 'FN': 1, 'threshold': 0.18752465024590492, 'latency': 0.0, 'reg_level': 0}
Results using peak-over-threshold method:
 {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'TP': 0, 'TN': 5566, 'FP': 1255, 'FN': 1, 'threshold': 0.23012295167414581, 'latency': 0.0}
Results using best f1 score search:
 {'f1': 0.00030106578307833426, 'precision': 0.00015055706089949254, 'recall': 0.9999900000999989, 'TP': 1, 'TN': 180, 'FP': 6641, 'FN': 0, 'threshold': 0.06970000000000001, 'latency': 0.0}
auc score : 0.04559448761178708

Saving output to output/IVECO/12012024_145138/<train/test>_output.pkl


/root/mtad-gat-pytorch/prediction.py:206: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_pred_df["A_True_Global"] = true_anomalies
/root/mtad-gat-pytorch/prediction.py:207: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_pred_df["Thresh_Global"] = global_epsilon
/root/mtad-gat-pytorch/prediction.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To g

-- Done.
